### **Lab 4: Partial Pivoting in Gaussian Elimination**

#### **1. Swapping Rows in Python**
Define the matrix:

$ A = \begin{bmatrix}2 & 1 & 2 \\ 3 & 5 & 2 \\ 7 & 4 & 4 \end{bmatrix} $  

We want to swap the first and third row. Our intuition tells us we can do this by defining a new variable (let's call it `holdRow`) that holds the first row, row 0. We can then set row 0 equal to row 2 and use our `holdRow` variable to assign row zero to row 2.

Let's try it. Print the matrix $A$ before and after the swap to see if it was swapped successfully.


In [59]:
import numpy as np

A = np.array([[2, 1, 2], [3, 5, 2], [7, 4, 4]])

print(f"Old A:\n{A}")

holdRow = A[0]
A[0] = A[2]
A[2] = holdRow

print(f"New A:\n{A}")

Old A:
[[2 1 2]
 [3 5 2]
 [7 4 4]]
New A:
[[7 4 4]
 [3 5 2]
 [7 4 4]]


**You may have noticed that it didn't do what we expected it to.**


#### **2. Debugging the Faulty Swap**

You should see that two of the rows are the same!

This is due to one of the ways that Python stores variables. When you assign a variable to be a collection of elements in an array, Python actually stores an alias of that array, storing the *address* of it instead of the actual values. This means that changes to our variable, i.e., changes to the array, will change the new variable as well.

In this case when we change the first row to be [7,4,4], we also inadvertently change `holdRow` to be [7,4,4] because `holdRow` is always equal to *the first row of A*, not actually the list [2,1,2].

To see this in action, copy the code from the previous section and print out `holdRow` right after it is assigned and right after the "swap" is perfomed.

In [60]:
A = np.array([[2, 1, 2], [3, 5, 2], [7, 4, 4]])

print(f"Old A:\n{A}")

holdRow = A[0,:]

print(f"holdRow before the swap: {holdRow}")

A[0, :] = A[2,:]
A[2, :] = holdRow

print(f"holdRow after the swap: {holdRow}")

print(f"New A:\n{A}")

Old A:
[[2 1 2]
 [3 5 2]
 [7 4 4]]
holdRow before the swap: [2 1 2]
holdRow after the swap: [7 4 4]
New A:
[[7 4 4]
 [3 5 2]
 [7 4 4]]


#### **3. Using .copy()**

The solution to this problem is quite simple. Create a copy of row 0 by adding `.copy()` to the end of the statement where we define `holdRow`. This assigns `holdRow` as a unique COPY of the first row of $A$ and therfore will not change when we change the rows of $A$.

So, let's try the swap again, but use `.copy()` this time.

In [61]:
A = np.array([[2, 1, 2], [3, 5, 2], [7, 4, 4]])

print(f"Old A:\n{A}")

holdRow = A[0,:].copy()

print(f"holdRow before the swap: {holdRow}")

A[0, :] = A[2,:]
A[2, :] = holdRow

print(f"holdRow after the swap: {holdRow}")

print(f"New A:\n{A}")

Old A:
[[2 1 2]
 [3 5 2]
 [7 4 4]]
holdRow before the swap: [2 1 2]
holdRow after the swap: [2 1 2]
New A:
[[7 4 4]
 [3 5 2]
 [2 1 2]]


#### **4. Finding the Maximum Value in a Column**

Now that we know how to swap rows, let's put this to good use.

When performing Gaussian elimination, dividing by the largest possible number can reduce roundoff error. Therefore we want to move the row containing the largest element in each column so that the largest number is on the diagonal. We do this by swapping the row with the one that currently sits at the pivot. Remember that negative values are valid too, so use the absolute value of the elements in the column to find the max.

Given the matrix $C$ below:

$ C = \begin{bmatrix}2 & 1 & 0 & 3 \\ 6 & 4 & 7 & 3 \\ 4 & 8 & 12 & 5 \\ 9 & 10 & 2 & 5\end{bmatrix} $

Use the NumPy function `argmax()` to find the *location* of the largest absolute value in the first column, then print the number of the row and the value of that element. Swap that row with the first row and print the new $C$.

In [62]:
C = np.array([[2, 1, 0, 3], [6, 4, 7, 3], [4, 8, 12, 5], [9, 10, 2, 5]])

maxLocation = np.argmax(abs(C[:, 0]))

print(f"Location of max in column 0 is: {maxLocation}")
print(f"The max value in column 0 is: {max(C[:, 0])}")

holdRow = C[:, 0].copy()
C[:, 0] = C[maxLocation, :]
C[maxLocation, :] = holdRow

print(f"The new C:\n{C}")

Location of max in column 0 is: 3
The max value in column 0 is: 9
The new C:
[[ 9  1  0  3]
 [10  4  7  3]
 [ 2  8 12  5]
 [ 2  6  4  9]]


#### **5. Adding Pivoting to Gaussian Elimination**

Now let's solve a problem with these methods. Take these four equations:

> $2x_0 + x_1+ 3x_4 = 9\\
6x_0 + 4x_1 + 7x_3 + 3x_4 = 5\\
4x_0 + 8x_1 + 12x_3 + 5x_4 = 4\\
9x_0 + 10x_1 + 2x_3 + 5x_4 = 1$

We want to find $x = [x_0, x_1, x_2, x_3]$, and we can do that very efficiently by using matrices. In this case, we will use:

$ C = \begin{bmatrix}2 & 1 & 0 & 3 \\ 6 & 4 & 7 & 3 \\ 4 & 8 & 12 & 5 \\ 9 & 10 & 2 & 5\end{bmatrix} B = \begin{bmatrix} 9 \\ 5 \\ 4\\ 1\end{bmatrix}$

Recalling matrix multiplication, we can find $x$ by solving $Cx=B$. We can do this with Gaussian elimination. The basic Gaussian elimination code from pg. 41 of the textbook is provided in the first code cell below, but we want to improve its accuracy by using partial pivoting.

In the second code cell, modify the function to pivot before the elimination of each column. You may copy/paste the code from the textbook or use your own Gaussian elimination code if you choose, but leave the original textbook function to be used for comparison.

When looking at the textbook code, answer these questions to better understand what to modify:

1.   What does `k` represent?
    K represents the 
2.   What does `i` represent?
3.   When do we need to swap rows?
4.   Where in the code does the row swapping go?
5.   In which rows should we look for the max value at each step?
6.   Do we need to swap the rows of $B$ as well?

Test your code against the textbook code using the matrix $C$ and the vector $B$. Once you find $x$, compute the residuals by computing $|Cx-B|$ and compare your result to the residuals you get from the non-pivoted example.

In [63]:

import numpy as np
# TEXTBOOK CODE - DO NOT ALTER

C = np.array([[2,1,0,3],[6,4,7,3],[4,8,12,5],[9,10,2,5]],dtype = float)
B = np.array([[9],[5],[4],[1]],dtype = float)

def gaussElimin(a,b):
  n = len(b)
  #Elimination Phase
  for k in range(0,n-1):
    for i in range(k+1,n):
      if a[i,k] != 0.0:
        lam = a[i,k]/a[k,k]
        a[i,k:n] = a[i,k:n] - lam*a[k,k:n]
        b[i] = b[i] - lam*b[k]
  #Back Substitution
  for k in range(n-1,-1,-1):
    b[k] = (b[k] - np.dot(a[k,k+1:n],b[k+1:n]))/a[k,k]
  return b

gauss = gaussElimin(C,B)
print('Final Solution = \n',gauss)

Final Solution = 
 [[ 0.30322581]
 [-1.90322581]
 [ 0.07096774]
 [ 3.43225806]]


In [64]:
# ADD PARTIAL PIVOTING HERE

def gaussEliminPartPivot(a,b):
  pass

print('Final Solution  =')

print("\nLet's make sure we get the same thing as before:\n")
print('Residuals for Pivoted\n')

print('\nResiduals for non-Pivoted\n')

Final Solution  =

Let's make sure we get the same thing as before:

Residuals for Pivoted


Residuals for non-Pivoted



#### **6. Scaled Partial Pivoting**

Roundoff error can be reduced even further if, when the matrix is pivoted, the pivot row is scaled so the largest element in the row is set to 1.

Copy the code above and modify it to divide each pivot row by its largest element.

Compare the residuals of this method to the residuals you got from the non-pivoted example and the regular partial pivoting solution.




In [65]:
import numpy as np

C = np.array([[2, 1, 0, 3], [6, 4, 7, 3], [4, 8, 12, 5], [9, 10, 2, 5]])

B = np.array([[9], [5], [4], [1]])

# ADD SCALING HERE

def gaussEliminScalePivot(a,b):
  n = len(b)
  #Elminitation Phase
  for k in range(0, n-1): 
    for i in range(k+1, n): 
      if a[i, k] != 0.0:
        lam = a[i, k]/a[k, k]
        a[i, k+1:n] = a[i, k+1:n] - lam*a[k, k+1:n]
        b[i] = b[i] - lam*b[k]
  #Back substitution
  for k in range(n-1, -1, -1):
    b[k] = (b[k] - np.dot(a[k, k+1:n], b[k+1:n]))/a[k, k]
  return b

gauss = gaussEliminScalePivot(C, B)
print(f"Final Solution = \n{gauss}")

print("\nLet's check the roundoff error again:\n")
print('Residuals for Scaled Pivoted\n')

print('\nResiduals for Pivoted\n')

print('\nResiduals for non-Pivoted\n')

#1. k means the diagonal, the pivot point
#2. i means the row
#3. right when you start considering the row
#4. once everything is 0 in the first column, all the rows below the first row are what matter
#5. 

NameError: name 'c' is not defined

#### **7. Why Scale or Pivot?**

For the previous problems we can see that there is very little difference between the different methods. Let's look at some cases where that is not the case. Use this new matrix $A$ (instead of $C$) with the same $B$ vector and again compare the residuals of the methods.

$A = \begin{bmatrix}600 & 0 & 1\times10^{15} & 13 \\ 1\times10^{13} & 5 & 0 & 7 \\ 1 & 3 & 5 & 17 \\ 1 & 1\times10^{-6} & 2 & 5\end{bmatrix} $

In [ ]:
A = np.array([[600,0,1e15,13],[1e13,5,0,7],[1,3,5,17],[1,1e-6,2,5]],dtype = float)
B = np.array([[9],[5],[4],[1]],dtype = float)

def gaussEliminPartPivot(a, b):
    

print('Residual for Scaled Pivoted\n')

print('\nResidual for Pivoted\n')

print('\nResidual for non-Pivoted\n')

Residual for Scaled Pivoted


Residual for Pivoted


Residual for non-Pivoted



Additional optional exercise: Convert the textbook code for Gaussian Elimination so that it utilizies an augmented matrix (i.e., if matrix $A$ is a matrix of size $n\times n$, then the Gaussian Elimination of $Ax = b$ could also be done by performing all of the row operations on some matrix $C$ of size $n \times (n+1)$, in which the last column is $b$.
